#### Import

In [1]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [3]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10", use_rt=True)
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1418


<Transform:
<Orientation: 
array([[ 0.00347748,  0.86583586, -0.50031607],
       [ 0.99998238, -0.00541818, -0.00242614],
       [-0.00481145, -0.50029882, -0.86583944]])>
<Vector: (-0.22654, 0.39427, 0.53016)>
>


In [3]:
from ddh import DDGripper
from scooping_primitives import HighSpeedScooping
gripper = DDGripper("ddh_scooping")
scoop = HighSpeedScooping(ur, gripper, "hss")

reading gripper config...
connecting to odrive...
found left finger
found right fingers
reading scooping config...


#### script testing

In [ ]:
gripper.arm()
gripper.set_stiffness(20)
gripper.set_left_tip((158, 38))
gripper.set_right_tip((158, -38))
# gripper.set_parallel_jaw(-14.5, 0)

In [22]:
ur.speedl([0.1,0,0,0,0,0],0.2,10)

In [23]:
print(ur.get_tcp_speed(wait=False))
ur.stopl()

[ 0.00054243 -0.09932496  0.00252892  0.00349176 -0.00432759 -0.00162943]


Robot not running: {'size': 47, 'type': 0, 'timestamp': 1700408000, 'isRobotConnected': True, 'isRealRobotEnabled': True, 'isPowerOnRobot': True, 'isEmergencyStopped': False, 'isSecurityStopped': False, 'isProgramRunning': False, 'isProgramPaused': False, 'robotMode': 4, 'controlMode': 0, 'speedFraction': 1.0, 'speedScaling': 0.0, 'speedFractionLimit': b'?'}
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/site-packages/urx/ursecmon.py", line 293, in run
    data = self._get_data()
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/site-packages/urx/ursecmon.py", line 340, in _get_data
    tmp = self._s_secondary.recv(1024)
socket.timeout: timed out



In [3]:
pose = ur.get_pose()
print(pose)

<Transform:
<Orientation: 
array([[ 6.29828511e-05,  8.66015589e-01, -5.00016995e-01],
       [ 9.99999997e-01, -7.69744017e-05, -7.35611181e-06],
       [-4.48590165e-05, -5.00016993e-01, -8.66015592e-01]])>
<Vector: (-0.21373, 0.39359, 0.52880)>
>


#### scooping test

In [4]:
scoop.initialize_gripper_pose((-0.509,0.45,180))
time.sleep(0.5)
a2_init = gripper.right_a2

Setting left tip: (150.0, 35.0)
Setting right tip: (128.5, 2.239092362730858)
Setting pose: 
[-0.5219      0.45        0.2        -1.92687451 -1.92687451  0.51630447]


In [113]:
# smack and scoop
smack_vel = 100 #100
smack_acc = 0.1 #0.3
stop_smack_acc = 4 #7
reverse_vel = 100 # 250
reverse_acc = 0.5 #1
stop_acc = 0.5 #0.5
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)
# robot detect collision, then stop, close gripper, and go up
while 1:
    a2_cur = gripper.right_a2
    #print(a2_cur)
    if a2_cur - a2_init > 0.3:
        print ("Collision detected!")
        gripper.set_stiffness(20, 'R')
        gripper.set_left_tip((158, 38))
#         gripper.set_left_tip((150, 40))
        gripper.set_right_tip((158, -38))
        ur.stopl(stop_smack_acc)
        gripper.set_stiffness(60, 'R')
        ur.speedl([0,0,reverse_vel,0,0,0],reverse_acc,5)
        time.sleep(0.3)
        ur.stopl(stop_acc)
        break

Collision detected!
Setting left tip: (158, 38)
Setting right tip: (158, -38)


In [ ]:
ur.stopl()